**Protkit Data Download Demo**

This short notebook illustrates the following:

*   Downloading PDB files from the Protein Data Bank.
*   Creating a Prot file containing the PDB files.
*   Visualising the proteins in 3D.

Protkit is available as a package on PyPI.

We start by installing Protkit and 3DMol. It could take a few minutes to install the necessary dependencies.

In [1]:
!pip install protkit
!pip install py3Dmol

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.1/270.1 kB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.9/206.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.5/575.5 kB 31.9 MB/s eta 0:00:00
  Created wheel for freesasa: filename=freesasa-2.2.1-cp310-cp310-linux_x86_64.whl size=853071 sha256=39f348cedae873ec626c2c4b5905011daf06b7dc64f27c2f82f4de169efd6ab8
  Stored in directory: /root/.cache/pip/wheels/ef/f5/89/77b77735ac804410dd420a5a8ed504321ad051cf65dd3d1f9f
Successfully built freesasa
  

Protkit contains a number of modules organised by the functionality they provide. In this demo, we are interested in downloading and file handling capabilties. To use these, we import the relevant modules and classes.

In [2]:
from protkit.download import Download
from protkit.file_io import PDBIO, ProtIO
import py3Dmol

For convenience, we list the proteins PDB ids we will use in this demo.

In [3]:
pdb_ids = ["1ahw", "1a4y", "1a6m", "1dc9"]

We can download all of PDB files from the Protein Data Bank through a single line of code. Protkit will download the files efficiently in parallel and store them in the data directory.

In [4]:
Download.download_pdb_files_from_rcsb(pdb_ids, "data")

We can load PDB files through the PDBIO class. The next line of code will load all the proteins into memory. They are stored as Protein objects, which expose a large number of useful functions that can be applied to proteins.

In [5]:
proteins = [PDBIO.load(f"data/{pdb_id}.pdb")[0] for pdb_id in pdb_ids]

Let's visualise the proteins. For this, we can 3DMol. 3DMol has capabilities to load proteins from file, but in this demo, we will create the proteins from our in-memory data representations.

In [6]:
viewer = py3Dmol.view(linked=False,viewergrid=(2, 2))
for i, protein in enumerate(proteins):
  viewer.addModel(PDBIO.save_to_string(proteins[i]),'pdb',viewer=(i // 2, i % 2))
  viewer.setStyle({'cartoon': {'color':'spectrum'}}, viewer=(i // 2, i % 2))
  viewer.zoomTo(viewer=(i // 2, i % 2))
viewer.render()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Let's save all our proteins in a single Prot file. This could be useful if we want to work with the data again in future.

In [7]:
ProtIO.save(proteins, "data/proteins.prot")

Prot files are stored using an efficient compressed JSON format. If we compare the file size of the database file just created, it is about half the size of a single PDB file.

In [9]:
import os

file_stats = os.stat("data/proteins.prot")
print(f'Database size in bytes: {file_stats.st_size}')

file_stats_pdb = os.stat("data/1ahw.pdb")
print(f'Single protein size in bytes: {file_stats_pdb.st_size}')

Database size in bytes: 423593
Single protein size in bytes: 862245
